In [18]:
import re
import string
import numpy as np
from nltk.tokenize import word_tokenize
import pandas as pd

In [54]:
diag_1 = "MetricsDB tem 3 componentes principais. O Cluster Manager é responsável por atribuir partições a servidores backend. HDFS é usado para armazenar mapeamentos de partições para servidores. Os servidores de back-end são responsáveis ​​pelo processamento de métricas para um pequeno número de partições. Cada servidor de back-end mantém as últimas duas horas de dados para todas as métricas na memória e verifica os dados na memória a cada duas horas para armazenamento durável, Blobstore. Os coordenadores são responsáveis ​​por rotear solicitações para todos os conjuntos de réplicas e valida que as restrições de quorum desejadas sejam atendidas. MetricsDB é compatível com várias zonas. MetricsDB tem um fator de replicação de três. O produtor Apache Kafka tem uma opção para solicitações em lote, o que nos ajudou a reduzir o número de solicitações para a fila e armazenamento."
#Resp 12, 13,25,26 and 49:
doc_12 = "O Coordinator vai enviar as métricas para o Kafka. O Kafka possui 3 réplicas, cada uma delas contendo 1 Cluster manager e um Backend Server O Cluster manager vai trabalhar com os HDFS enquanto o Backend vai alimentar o Blobstore"
doc_13 = "Arquitetura de integrações baseada em 4 camadas, onde, em uma delas, o Kafka recebe dados de um Coordinator e os envia a um servidor com 3 brokers, que, por sua vez, alimenta os sistemas HDFS e Blobstore."
doc_25 = "Informações que são coletadas são enviadas para clusters ou particões através do intermédio do Apache Kafta."
doc_26 = "Existem uma série de Replicas disponíveis para atendimento das Necessidades recebidas de Kafka. Cluster Manager é responsável por HDFS e Backend Servers responsável por Blobsotre"

diag_2 = "Sempre que ocorre um evento no cliente do Spotify, ele é enviado para um dos gateways do Spotify, que o registra via syslog. Lá, é atribuído um carimbo de data / hora que é usado em todo o sistema de entrega de eventos. todos os dados precisam ser entregues a um cluster Hadoop localizado centralmente. Todos os dados entregues por meio de nosso serviço de entrega de eventos são gravados em HDFS. O File Tailer controla os arquivos de log em busca de novos eventos e os encaminha para o Event Delivery Service. Assim que obtiver a confirmação de que o evento foi recebido, a sua responsabilidade termina. O Event Delivery Service aceita eventos do Tailer, transforma-os em seu formato estruturado final e os encaminha para os Kafka Brokers. Event Delivery Service é construído como um microsserviço RESTful usando a estrutura Apollo e implementado usando a plataforma de orquestração Helios, um padrão de design comum no Spotify. Incorporar um produtor Kafka simples no serviço de entrega de eventos também provou ser fácil. O projeto Mirror Maker introduziu o espelhamento entre data centers e o projeto Camus pode ser usado para exportar eventos estruturados Avro para depósitos de hora em hora."
diag_3 = "DBLog é um framework genérico de Captura de Dados de Mudança (CDC)  que permite capturar mudanças confirmadas de um banco de dados em tempo real e propagar essas mudanças para consumidores downstream. Os logs de transações de bancos de dados são a origem dos eventos do CDC. DBLog é uma estrutura baseada em Java, capaz de capturar mudanças em tempo real e fazer dumps. Usamos o Zookeeper para armazenar o estado relacionado ao processamento de registro e dump e para a eleição do líder. Para usar DBLog, um banco de dados precisa fornecer um log de alterações a partir de um histórico linear de alterações confirmadas e essas condições são atendidas por sistemas como MySQL, PostgreSQL, MariaDB, etc. O processamento de dumps foi integrado ao sistema usando SQL e JDBC, exigindo apenas a implementação de seleção do bloco e a atualização da marca d'água. O mesmo código é usado para MySQL e PostgreSQL e também pode ser usado para outros bancos de dados semelhantes. DBLog usa uma arquitetura ativa-passiva. Uma instância está ativa e as outras estão em espera passiva. Aproveitamos o Zookeeper para a eleição do líder para determinar a instância ativa."

corpus = np.array([diag_1, doc_12, doc_13, doc_25, doc_26, diag_2, diag_3])

### 1) Aplicar lowercase e remover caracteres

In [55]:
def clear_text(text):
    pattern = "[{}]".format(string.punctuation)
    text = [word.lower() for word in text]
    text = [[re.sub(pattern, "", word) for word in words.split()] for words in text]
    text = [[word for word in words if len(word)>1] for words in text]    
    text = [' '.join(words) for words in text]
    return np.array(text)

In [56]:
corpus_clear = clear_text(corpus)
print(corpus_clear)

['metricsdb tem componentes principais cluster manager responsável por atribuir partições servidores backend hdfs usado para armazenar mapeamentos de partições para servidores os servidores de backend são responsáveis \u200b\u200bpelo processamento de métricas para um pequeno número de partições cada servidor de backend mantém as últimas duas horas de dados para todas as métricas na memória verifica os dados na memória cada duas horas para armazenamento durável blobstore os coordenadores são responsáveis \u200b\u200bpor rotear solicitações para todos os conjuntos de réplicas valida que as restrições de quorum desejadas sejam atendidas metricsdb compatível com várias zonas metricsdb tem um fator de replicação de três produtor apache kafka tem uma opção para solicitações em lote que nos ajudou reduzir número de solicitações para fila armazenamento'
 'coordinator vai enviar as métricas para kafka kafka possui réplicas cada uma delas contendo cluster manager um backend server cluster manag

### 2) Remover stopwords

In [57]:
from nltk.corpus import stopwords
def clear_stopwords(text, language):
    filtered_text = []
    for t in text:
        words = [w for w in t.split() if not w in stopwords.words(language)]
        words = ' '.join(words)
        filtered_text.append(words)
    return np.array(filtered_text)   

In [58]:
corpus_filtered = clear_stopwords(corpus_clear, "english")
print(corpus_filtered)

['metricsdb tem componentes principais cluster manager responsável por atribuir partições servidores backend hdfs usado para armazenar mapeamentos de partições para servidores os servidores de backend são responsáveis \u200b\u200bpelo processamento de métricas para um pequeno número de partições cada servidor de backend mantém últimas duas horas de dados para todas métricas na memória verifica os dados na memória cada duas horas para armazenamento durável blobstore os coordenadores são responsáveis \u200b\u200bpor rotear solicitações para todos os conjuntos de réplicas valida que restrições de quorum desejadas sejam atendidas metricsdb compatível com várias zonas metricsdb tem um fator de replicação de três produtor apache kafka tem uma opção para solicitações em lote que nos ajudou reduzir número de solicitações para fila armazenamento'
 'coordinator vai enviar métricas para kafka kafka possui réplicas cada uma delas contendo cluster manager um backend server cluster manager vai traba

## Usando a biblioteca sklearn

In [59]:
def create_dataframe(matrix, tokens):
    doc_names = [f'text_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorize = CountVectorizer()
vector_matrix = vectorize.fit_transform(corpus_filtered)

tokens = vectorize.get_feature_names()
create_dataframe(vector_matrix.toarray(),tokens)



/home/jalves/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aceita,ajudou,alimenta,alimentar,alterações,ao,apache,apenas,apollo,aproveitamos,...,usar,vai,valida,verifica,vez,via,várias,zonas,zookeeper,últimas
text_1,0,1,0,0,0,0,1,0,0,0,...,0,0,1,1,0,0,1,1,0,1
text_2,0,0,0,1,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
text_3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
text_4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
text_5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
text_6,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
text_7,0,0,0,0,2,2,0,1,0,1,...,1,0,0,0,0,0,0,0,2,0


In [61]:
cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,['Diag_1','resp_1','resp_2', 'resp_3', 'resp_4','Diag_2', 'Diag_3'])

,Diag_1,resp_1,resp_2,resp_3,resp_4,Diag_2,Diag_3
text_1,1.000000,0.278093,0.405096,0.197642,0.430628,0.577251,0.632167
text_2,0.278093,1.000000,0.289683,0.037689,0.307941,0.156360,0.103852
text_3,0.405096,0.289683,1.000000,0.040032,0.267615,0.511529,0.434744
text_4,0.197642,0.037689,0.040032,1.000000,0.046424,0.124461,0.131697
text_5,0.430628,0.307941,0.267615,0.046424,1.000000,0.300454,0.353663
text_6,0.577251,0.156360,0.511529,0.124461,0.300454,1.000000,0.638832
text_7,0.632167,0.103852,0.434744,0.131697,0.353663,0.638832,1.000000
